In [ ]:
import string
import random
import string
import pickle
from string import ascii_letters
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import sqlite3
from datetime import datetime,timedelta
import pandas as pd
from chatbot_respuesta import *

CREAR TOKEN

In [ ]:
scopes=['https://www.googleapis.com/auth/calendar']
flow=InstalledAppFlow.from_client_secrets_file('CREDENTIAL.json',scopes=scopes)

In [ ]:
credentials=pickle.load(open('token.pkl','rb'))
service=build('calendar','v3',credentials=credentials)

In [ ]:
##Crear bases de datos

import sqlite3


conn = sqlite3.connect('Calendario_citas.db')

cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='Citas'")
if cursor.fetchone() is None:
    # Create the COMPANY table
    cursor.execute('''CREATE TABLE Citas
                    (id_cita TEXT NOT NULL,
                    calendar_id TEXT NOT NULL,
                    servicio TEXT NOT NULL,
                    nombre TEXT NOT NULL,
                    description TEXT NOT NULL,
                    hora_de_inicio INTEGER,
                    hora_finalizacion INTEGER,
                    correo TEXT NOT NULL);''')

    print("Bases de datos creado")
else:
    print("Table already exists")
conn.close()

PROCESO DE AGENDAR CITAS

In [ ]:
def range_alpha(start_letter, end_letter):
    return ascii_letters[
    ascii_letters.index(start_letter):ascii_letters.index(end_letter) + 1
  ]

def codigo_cita():
    letras=range_alpha('a', 'v')
    characters = letras + string.digits 
    cita_id = ''.join(random.choice(characters) for i in range(8))
    return cita_id+'5'


def mostrarDatos(dictinfo, msgconf):
    print(msgconf,'\n')
    for key in dictinfo.keys():    
        print(key, ':', dictinfo[key])

In [ ]:
def getInfo(msgInicial, listaConsultas, servicio):
    print(msgInicial)
    for i, pregChat in enumerate(listaConsultas):
        print(pregChat)
        entradaUsuario = input()
        if i ==0:
            nombre = entradaUsuario
            
        if i ==1:
            fecha = str(entradaUsuario+' ')
            
        if i ==2:
            hora = str(entradaUsuario+':00')
            fecha= fecha+hora
            start_time= datetime.strptime(fecha, "%d/%m/%Y %H:%M:%S")
            end_time=start_time+timedelta(hours=1)

        if i ==3:
            correo = entradaUsuario  
        if i ==4:
            description = entradaUsuario 

    return {'nombre':nombre, 'descripcion':description, 'hora_de_inicio':start_time, 'hora_finalización':end_time,'correo':correo}
            
msgInicial = "Para consultar disponibilidad indiquenos los siguientes datos:"
listaConsultas = ['por favor indiqueme el nombre','Ingrese fecha en formato *DD/MM/AAAA* ','Ingrese hora en formato de 24 horas *hora:min*','Ingrese su correo electrónico','Porqué está agendando esta cita?']


In [ ]:
def crear_evento(nombre,servicio,id_cita,description,start_time,end_time,correo,calendar_id):
    event = { 
        'summary': (nombre + '  servicio: ' + servicio),
        'id':id_cita,
          'location': 'Local NO. 3,Town Center, Costa del Este, Ciudad de Panamá',
          'description': description,
          'start': {
            'dateTime': start_time.strftime('%Y-%m-%dT%H:%M:%S'),
            'timeZone': 'America/Panama',
          },
          'end': {
            'dateTime': end_time.strftime('%Y-%m-%dT%H:%M:%S'),
            'timeZone': 'America/Panama',
          },
          'attendees': [
            {'email': correo}
          ],
          'reminders': {
            'useDefault': False,
            'overrides': [
              {'method': 'email', 'minutes': 24 * 60},
              {'method': 'popup', 'minutes': 10},
            ],
          },
        }
    service.events().insert(calendarId=calendar_id, body=event).execute()

In [ ]:
def aggdatos_sql(datos):
    conn = sqlite3.connect('Calendario_citas.db')
    cursor = conn.cursor()
    ingresar= cursor.execute("INSERT INTO Citas VALUES (:id_cita, :calendar_id,:servicio, :nombre,:descripcion,:hora_de_inicio,:hora_finalización,:correo)", infoCita) 
    conn.commit()
    conn.close()
    return ingresar

In [ ]:
def confirmar_datos(intencion,infoCita,servicio,calendar_id):
    if intencion=='Afirmación' :
            id_citas= codigo_cita()
            infoCita['id_cita']=id_citas
            infoCita['calendar_id']=calendar_id
            crear_evento(infoCita['nombre'],servicio,id_citas,infoCita['descripcion'],infoCita['hora_de_inicio'],infoCita['hora_finalización'],infoCita['correo'],infoCita['calendar_id'])
            aggdatos_sql(infoCita)
            print('Su cita ha sido agendada con éxito, en que más puedo ayudarte?, su id de cita es: ',id_citas )
    elif intencion=='Negación':
            print('Ok, cita no agendada. En que más puedo ayudarte?')


In [ ]:
id_asesoria='longstoryshort51@gmail.com'
id_atencióncliente='1e8f7a6f98f1ef21fe6c91c81f4bf84ea6715d776b3621517fb74a04fec2badd@group.calendar.google.com'

PRUEBA

In [ ]:
entradaUsuario = ''
intencion_previa = ''
msgconf = 'Para terminar el proceso de agendar su cita, confirme que los datos ingresados son correctos.'

while entradaUsuario != 'exit':
    entradaUsuario = input()
    respuesta, intencion = chatbot_respuesta(entradaUsuario)
  
    respuesta, intencion = chatbot_respuesta(entradaUsuario)  
    if intencion_previa =='Asesoría' and entradaUsuario == '1':
                servicio='Asesoría'
                calendar_id=id_asesoria
                infoCita = getInfo(msgInicial, listaConsultas, servicio)
                infoCita['servicio']=servicio
                mostrarDatos(infoCita, msgconf)
                entradaUsuario = input()
                respuesta, intencion = chatbot_respuesta(entradaUsuario)
                confirmar_datos(intencion,infoCita,servicio,calendar_id)


    elif intencion_previa =='Agendar cita' and entradaUsuario == '2':
                servicio='Asesoría'
                calendar_id=id_atencióncliente
                infoCita = getInfo(msgInicial, listaConsultas, servicio)
                infoCita['servicio']=servicio
                mostrarDatos(infoCita, msgconf)
                entradaUsuario = input()
                respuesta, intencion = chatbot_respuesta(entradaUsuario)
                confirmar_datos(intencion,infoCita,servicio,calendar_id)
     
        
    intencion_previa = intencion  
    print(respuesta) 
